# 1. Install jsonlines, Numpy, beir
```
pip install jsonlines numpy beir


# 2. Triviaqa Dataset Download
```
wget http://nlp.cs.washington.edu/triviaqa/data/triviaqa-rc.tar.gz
tar -xvf triviaqa-rc.tar.gz

# 3. Import Needed Libraries

In [1]:
from tqdm import tqdm
import jsonlines
import numpy as np
import os
import json
from beir.datasets.data_loader import GenericDataLoader

# 4. Main Functions

### retrieve_eval_data_loader()

In [ ]:
def retrieve_eval_data_loader(data_name, query_file, corpus_file,qrels_file):

    data_path = f"datasets/{data_name}/"
    
    print("=====> Loading QA Dataset")
    print("=====> Loading Pool of Documents")
    corpus, queries, qrels = GenericDataLoader(
        data_folder = data_path,\
        corpus_file = corpus_file,\
        query_file = query_file,\
        ).load(split=qrels_file) # or split = "train" or "dev"
    

    
    return corpus, queries, qrels

### custom_data_loaer()

In [ ]:
def custom_data_loader(args):

    if args.data_name == 'triviaqa':
        questions = []
        answers = []
        with jsonlines.open(args.qa_data) as f:
            for line in f.iter():
                questions.append(line["query"])
                answers.append(line["answers"])
    
        retrieval_queries = {}
        for i in tqdm(range(len(questions)), desc="QA Data Preprocessing"):
            question = questions[i]
            qa_id = str(args.data_name) + "_" + str(i)
            retrieval_queries[qa_id] = question
    
        titles = []
        texts = []
        with jsonlines.open(args.documents_pool) as f:
            for line in f.iter():
                texts.append(line["text"])
                titles.append(line["title"])
    
        retrieval_corpus = {}
        for i in tqdm(range(len(titles)), desc= "Documents_Pool Preprocessing"):
            json_obj = {}
            json_obj["title"] = titles[i]
            json_obj["text"] = texts[i]
            retrieval_corpus[str(i)] = json_obj

    return retrieval_queries, retrieval_corpus, questions, answers, titles, texts

### make_new_dataset()

In [ ]:
def make_new_dataset(args, retrieved_doc):
        
        _,_, questions, answers, titles,texts = custom_data_loader(args)
        print("=====> Starting Construction of Dataset")

        sorted_idxs = []
        sorted_scores = []

        for i in range(len(titles)):
            scores_i = np.array(list(retrieved_doc['{}_{}'.format(args.data_name, i)].values()))
            sorted_idx = np.argsort(scores_i)[::-1]
            keys = list(retrieved_doc['{}_{}'.format(args.data_name, i)].keys())

            sorted_idxs_i = []
            sorted_scores_i = []
            for j in range(min(len(scores_i), args.num_retrieval)):
                sorted_idxs_i.append(int(keys[sorted_idx[j]]))
                sorted_scores_i.append(scores_i[sorted_idx[j]])

            sorted_idxs.append(sorted_idxs_i)
            sorted_scores.append(sorted_scores_i)

        res = []
        for i in range(len(questions)):
            new_item = {}
            new_item['question'] = questions[i]
            new_item['answer'] = answers[i]

            ctxs = []
            for j in range(len(sorted_idxs[i])):
                ctx = {}
                ctx['id'] = sorted_idxs[i][j]
                ctx['title'] = titles[sorted_idxs[i][j]]
                ctx['text'] = texts[sorted_idxs[i][j]]
                ctx['score'] = sorted_scores[i][j]
                ctxs.append(ctx)
            new_item['contexts'] = ctxs
            res.append(new_item)

        if not os.path.exists(args.output_folder):
            os.makedirs(args.output_folder)
        
        print("=====> All Procedure is finished!")
        with open(f"{args.output_folder}/{args.data_name}/{args.retrieval_method}_retrieved_docs.doc",'w') as writer:
            writer.write(json.dumps(res, indent=4, ensure_ascii=False) + "\n")

# 5. Run

In [ ]:
class Args:
    data_name = 'triviaqa'
    qa_data = '/content/qa/wikipedia-dev.json'  # TriviaQA QA 데이터 경로
    documents_pool = '/content/documents_pool.json'  # 문서 풀 데이터 경로
    output_folder = 'output_folder'
    retrieval_method = 'retrieval_method'
    num_retrieval = 5

args = Args()

In [ ]:
retrieved_doc = {f'triviaqa_{i}': {str(j): np.random.random() for j in range(10)} for i in range(5)}  # 예시 데이터

make_new_dataset(args, retrieved_doc)